In [ ]:
# WhipperSnapPy Demo - Static & Interactive Rendering
# This notebook demonstrates both static and interactive 3D brain visualization.
#
# Installation:
# pip install 'whippersnappy[notebook]'


In [ ]:
import os

from IPython.display import display

from whippersnappy import ViewType, plot3d, snap1, snap4


In [ ]:
# Setup Paths
# Edit these to point to your FreeSurfer/FastSurfer data:

# Set your subject directory here
# (either an absolute path to a subject's directory
# containing a `surf/` subdirectory).
# Example: sdir = '/home/user/freesurfer/subjects/subject01'
# IMPORTANT: set this value before running the rest of the notebook.
sdir = '/path/to/your/subjectdir'  # <-- update

# Verify that sdir exists and is a directory
if not os.path.isdir(sdir):
    raise ValueError(
        f"Subject directory does not exist: {sdir}\n"
        "Please set `sdir` to a valid subject directory containing a 'surf/' subdirectory."
    )

# Derive per-hemisphere paths from the subject directory
lh_surf_path = os.path.join(sdir, 'surf', 'lh.white')
lh_thickness_path = os.path.join(sdir, 'surf', 'lh.thickness')
lh_curv_path = os.path.join(sdir, 'surf', 'lh.curv')
lh_label_path = os.path.join(sdir, 'label', 'lh.cortex.label')
lh_annot_path = os.path.join(sdir, 'label', 'lh.aparc.annot')

rh_surf_path = os.path.join(sdir, 'surf', 'rh.white')
rh_thickness_path = os.path.join(sdir, 'surf', 'rh.thickness')
rh_curv_path = os.path.join(sdir, 'surf', 'rh.curv')
rh_label_path = os.path.join(sdir, 'label', 'rh.cortex.label')
rh_annot_path = os.path.join(sdir, 'label', 'rh.aparc.annot')

# Preset overlay variables for convenience for Snap4
lh_overlay = lh_thickness_path if os.path.exists(lh_thickness_path) else None
rh_overlay = rh_thickness_path if os.path.exists(rh_thickness_path) else None

print(f"Subject dir: {sdir}")
print(f"Surface exists?   LH: {os.path.exists(lh_surf_path)} | RH: {os.path.exists(rh_surf_path)}")
print(f"Thickness exists? LH: {os.path.exists(lh_thickness_path)} | RH: {os.path.exists(rh_thickness_path)}")
print(f"Curv exists?      LH: {os.path.exists(lh_curv_path)} | RH: {os.path.exists(rh_curv_path)}")
print(f"Label exists?     LH: {os.path.exists(lh_label_path)} | RH: {os.path.exists(rh_label_path)}")
print(f"Annot exists?     LH: {os.path.exists(lh_thickness_path)} | RH: {os.path.exists(rh_thickness_path)}")



In [ ]:
# Part 1: Static Rendering - Single View
# Generate publication-quality static images with full PyOpenGL control.

# Render a single view
img = snap1(
    meshpath=lh_surf_path,
    overlaypath=lh_overlay,
    curvpath=lh_curv_path if os.path.exists(lh_curv_path) else None,
    view=ViewType.LEFT,
    width=800,
    height=800,
    brain_scale=1.5,  # Adjust to avoid cropping
    specular=True,    # Professional lighting
)
display(img)


In [ ]:
# Part 1: Static Rendering - Snap4 (both hemispheres)
# Use snap4 to render front/back views for left and right
# hemispheres and display the combined image.

print(f"Using subjects dir: {sdir}")

# lh_overlay and rh_overlay are precomputed in the Setup cell
# Call snap4 and receive a PIL Image directly when outpath=None
img4 = snap4(
    lhoverlaypath=lh_overlay,
    rhoverlaypath=rh_overlay,
    sdir=sdir,
    caption='Snap4 - both hemispheres',
    outpath=None,  # return PIL image instead of writing to disk
    specular=True,
    brain_scale=1.8,
)

# Display result (snap4 returns a PIL.Image when outpath is None)
if img4 is not None:
    display(img4)
else:
    print("snap4 did not return an image; check inputs and OpenGL context.")


In [ ]:
# Part 2: Interactive 3D Rendering
# Mouse-controlled 3D visualization using Three.js (works in all Jupyter environments).
#
# Controls:
# - Drag: Rotate
# - Scroll: Zoom
# - Right-drag: Pan

# Interactive viewer with curvature (grayscale)
viewer = plot3d(
    meshpath=lh_surf_path,
    curvpath=lh_curv_path if os.path.exists(lh_curv_path) else None,
    width=800,
    height=800,
)
display(viewer)


In [ ]:
# Interactive colored overlay (if available)
if lh_overlay:
    viewer = plot3d(
        meshpath=lh_surf_path,
        overlaypath=lh_overlay,
        curvpath=lh_curv_path if os.path.exists(lh_curv_path) else None,
        labelpath=lh_label_path,
        minval=0.0,  # Threshold
        maxval=5.5,  # Saturation
        width=800,
        height=800,
    )
    display(viewer)
else:
    print("Thickness overlay not found - skipping colored example")


In [ ]:
# Interactive label map overlay
if lh_annot_path:
    viewer = plot3d(
        meshpath=lh_surf_path,
        annotpath=lh_annot_path,
        curvpath=lh_curv_path if os.path.exists(lh_curv_path) else None,
        width=800,
        height=800,
    )
    display(viewer)
else:
    print("Annot overlay not found - skipping label map example")


In [ ]:
# Notes
#
# Static Rendering:
# - Returns PIL Image objects (no disk I/O needed)
# - Full PyOpenGL control for custom lighting
# - Publication-quality output
# - Fast and deterministic
#
# Interactive Rendering:
# - Uses Three.js/WebGL (runs in browser)
# - Works in all Jupyter environments
# - Full mouse control (rotate, zoom, pan)
# - Same technology as Plotly 3D plots
#
# Color Notes:
# - Curvature: Grayscale (sulci = dark, gyri = light) - this is correct!
# - Overlays: Colored heatmaps (thickness, activation, statistics)
# - Annotations: Distinct colored regions (parcellations)
